#### Student Name: Dwight Devens
#### Student ID: A15711217

# CSE 190 Final Project

### Suggested Project 1

### Composing Melody Using RNN with Attention

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Lambda
from keras_sequential_ascii import keras2ascii

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
numberOfNotesForPrediction = 8
numberOfEpochsToTrainFor = 5

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Parses each midi file and extracts the individual notes
def parseNotes():

    notesAlreadyParsed = []
    # For each midi file
    for file in glob.glob("MyData/*.mid"):
        # Reset the note counter
        noteCounter = None
        # Parse the file
        midi = converter.parse(file)
        # If the file has instruments in it
        try:
            checkInstrument = instrument.partitionByInstrument(midi)
            noteCounter = checkInstrument.parts[0].recurse()
        # And if that fails
        except:
            noteCounter = midi.flat.notes
        # Then for each note
        for thisNote in noteCounter:
            # If it is a chord
            if isinstance(thisNote, chord.Chord):
                notesAlreadyParsed.append('.'.join(str(n) for n in element.normalOrder))
            # Else it is a note
            elif isinstance(thisNote, note.Note):
                notesAlreadyParsed.append(str(thisNote.pitch))
    
    # Done with files, open output and write it
    with open('MyData/notes', 'wb') as filepath:
        pickle.dump(notesAlreadyParsed, filepath)
    return notesAlreadyParsed

In [3]:
def constructMelodies(notesAlreadyParsed, n_vocab, numberOfNotesforPrediction):
    
    # Sort the pitches
    sortedPitches = sorted(set(item for item in notesAlreadyParsed))
    # Map them to ints
    noteIntegerDict = dict((note, number) for number, note in enumerate(sortedPitches))
    networkInput = []
    networkOutput = []
    
    # Create the input/output to the network
    for i in range(0, len(notesAlreadyParsed) - numberOfNotesforPrediction, 1):
        # Input
        inputSequence = notesAlreadyParsed[i:i + numberOfNotesforPrediction]
        networkInput.append([noteIntegerDict[char] for char in inputSequence])
        # Output
        outputSequence = notesAlreadyParsed[i + numberOfNotesforPrediction]
        networkOutput.append(noteIntegerDict[outputSequence])

    lengthOfNetwork = len(networkInput)

    # Had to change shape for LSTM layers or it wasn't working
    networkInput = np.reshape(networkInput, (lengthOfNetwork, numberOfNotesforPrediction, 1))
    networkInput = networkInput / float(n_vocab)
    # Other utils wasn't working leave this with current versions
    networkOutput = np_utils.to_categorical(networkOutput)
    return (networkInput, networkOutput)

In [4]:
def constructNetwork(networkInput, n_vocab):

    model = Sequential()
    model.add(LSTM(512, input_shape=(networkInput.shape[1], networkInput.shape[2]),
                   recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    
    # 2. A batch normalization layer. 
    # Keras documentation - "Layer that normalizes its inputs. Batch normalization 
    # applies a transformation that maintains the mean output close to 0 and the output 
    # standard deviation close to 1.""
    # Example: Args look optional, just instantiate for now
    model.add(BatchNorm())
    
    # 3. A layer which drops 3/10 of the units. 
    # Keras documentation - "The Dropout layer randomly sets input units to 0 with a 
    # frequency of rate at each step during training time, 
    # which helps prevent overfitting. 
    # Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all 
    # inputs is unchanged.""
    # Example: tf.keras.layers.Dropout(rate, noise_shape=None, seed=None, **kwargs)
    model.add(Dropout(0.3))
    
    # 4. A fully connected layer with 256 units of output. 
    # Keras documentation - "Just your regular densely-connected NN layer."
    # Example: model.add(tf.keras.layers.Dense(32))
    model.add(Dense(256))
    
    # 5. A ReLU activation layer. 
    # Keras documentatino - "Applies an activation function to an output.
    # Arguments - activation: Activation function, such as tf.nn.relu, 
    # or string name of built-in activation function, such as "relu"."
    # Example: layer = tf.keras.layers.Activation('relu')
    model.add(Activation('relu'))
    
    # 6. A batch normalization layer. 
    # Keras documentation - "Layer that normalizes its inputs. Batch normalization 
    # applies a transformation that maintains the mean output close to 0 and the output 
    # standard deviation close to 1.""
    # Example: Args look optional, just instantiate for now
    model.add(BatchNorm())
    
    # 7. A layer which drops 3/10 of the units. 
    # Keras documentation - "The Dropout layer randomly sets input units to 0 with a 
    # frequency of rate at each step during training time, 
    # which helps prevent overfitting. 
    # Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all 
    # inputs is unchanged.""
    # Example: tf.keras.layers.Dropout(rate, noise_shape=None, seed=None, **kwargs)
    model.add(Dropout(0.3))
    
    # 8. A fully connected layer with number of units of output equal to the vocabulary 
    # space of the input. 
    # Keras documentation - "Just your regular densely-connected NN layer."
    # Example: model.add(tf.keras.layers.Dense(32))
    # but this time need vocab space of input, which = n_vocab  
    model.add(Dense(n_vocab))
    
    # 9. A softmax activation layer which uses a temperature of .6 
    # (Note, you may need to define this as two separate layers in Keras, 
    # using the definition of temperature for softmax)
    
    # Keras documentation - "Softmax converts a vector of values to a probability 
    # distribution. The elements of the output vector are in range (0, 1) and sum to 1.
    # Each vector is handled independently. The axis argument sets which axis of the 
    # input the function is applied along. Softmax is often used as the activation 
    # for the last layer of a classification network because the result could be 
    # interpreted as a probability distribution. The softmax of each vector x is 
    # computed as exp(x) / tf.reduce_sum(exp(x)). The input values in are the log-odds 
    # of the resulting probability.
    # Arguments - x : Input tensor. axis: Integer, axis along which the softmax 
    # normalization is applied."
    model.add(Activation('softmax'))
    
    # After creating your network, compile the model with categorical cross entropy loss 
    # and an optimizer of your choice. 
    # Example: model.compile(loss='categorical_crossentropy', optimizer=opt)
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(filePathToWeights)

    return model

In [5]:
def trainTheNetwork(numberOfNotesforPrediction, numberOfEpochsToTrainFor):
    # Parse the notes and store the amount
    notesAlreadyParsed = parseNotes()
    n_vocab = len(set(notesAlreadyParsed)) 
    # Construct the melodies
    networkInput, networkOutput = constructMelodies(notesAlreadyParsed, n_vocab, numberOfNotesforPrediction)
    # Build the network
    networkModel = buildTheNetwork(networkInput, n_vocab)
    # Train the network
    weightsFile = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = theModelCheckpoint(weightsFile, monitor='loss', verbose=0,
                                save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    networkModel.fit(networkInput, networkOutput, 
                 epochs=numberOfEpochsToTrainFor, batch_size=128, callbacks=callbacks_list)
    
    print("done training")    

In [ ]:
trainTheNetwork(numberOfNotesForPrediction, numberOfEpochsToTrainFor)
print("end")